In [1]:
import time 
import datetime
import numpy as np
import pandas as pd
import tweepy
from local_config import *

# assign the values accordingly 
consumer_key = cons_tok 
consumer_secret = cons_sec 
access_token = app_tok 
access_token_secret = app_sec
  
# authorization of consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
# set access to user's access key and access secret  
auth.set_access_token(access_token, access_token_secret) 
  
# calling the api  
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [2]:
list1 = ['enter list of hashtags or mentions to search for']

In [3]:
from pymongo import MongoClient
import json

client = MongoClient('insert your mongo db url')
db = client.tweepydb

In [4]:
#Below function could be used to make lookup requests for ids 100 at a time leading to 18K lookups in each 15 minute window
def get_usernames(userids, api):
    fullusers = []
    u_count = len(userids)
    try:
        for i in range(int(u_count/100) + 1):            
            end_loc = min((i + 1) * 100, u_count)
            fullusers.extend(
                api.lookup_users(user_ids=userids[i * 100:end_loc])                
            )
        return fullusers
    except:
        import traceback
        traceback.print_exc()
        print ('Something went wrong, quitting...')

In [5]:
dtfm = []

candidate = list1[0]
excel = pd.read_excel('followers_of_'+candidate+'.xlsx')

followerids =list(excel['followers'])

#Calling the function below with the list of followeids and tweepy api connection details
fullusers = get_usernames(followerids,api)

#create dataframe

for follower in fullusers:
    fw = follower._json
    fw['candidate'] = candidate
    dtfm.append(fw)
    db.followers.insert_one(fw)

In [5]:
db.followers.delete_many({'candidate':{'$ne':''}})

In [10]:
new_df.to_excel('full_followers'+candidate+'.xlsx')